<a href="https://colab.research.google.com/github/longevity-genie/just-agents/blob/main/examples/notebooks/02_sqlite_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Building your Own SQL Agent with Just-Agents ##


This notebook provides a comprehensive guide on how to create and utilize your own custom SQL agents using [just-agents](https://github.com/longevity-genie/just-agents/) library.

It is the second tutorial in a series of tutorials, next ones are:
1. Basic agents tutorial  
2. Database agent tutorial (THIS ONE)
3. Coding agent tutorial

## Working with the Open-Genes Database

For this tutorial, we'll be interacting with the Open-Genes database. We'll be using a simplified version stored in an SQLite file. You can access the original database file at:

[https://github.com/longevity-genie/longevity_gpts/blob/main/open_genes/data/open_genes.sqlite](https://github.com/longevity-genie/longevity_gpts/blob/main/open_genes/data/open_genes.sqlite)

Let's download the database first

In [1]:
!wget https://github.com/longevity-genie/longevity_gpts/raw/main/open_genes/data/open_genes.sqlite -O open_genes.sqlite

--2025-05-21 19:26:01--  https://github.com/longevity-genie/longevity_gpts/raw/main/open_genes/data/open_genes.sqlite
Resolving github.com (github.com)... 140.82.116.3
Connecting to github.com (github.com)|140.82.116.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/longevity-genie/longevity_gpts/main/open_genes/data/open_genes.sqlite [following]
--2025-05-21 19:26:01--  https://raw.githubusercontent.com/longevity-genie/longevity_gpts/main/open_genes/data/open_genes.sqlite
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1150976 (1.1M) [application/octet-stream]
Saving to: ‘open_genes.sqlite’

open_genes.sqlite   100%[===================>]   1.10M  --.-KB/s    in 0.04s   

2025-05-21 19:26:01 (2

Let's test the query by outputing the tables

In [2]:
import sqlite3

def db_query(sql: str):
    """ This function execute query on open-genes sqlite table. It returns query results. """
    conn = sqlite3.connect("open_genes.sqlite")
    cursor = conn.cursor()

    cursor.execute(sql)
    try:
        rows = cursor.fetchall()
        if rows is None or len(rows) == 0:
            conn.close()
            return ""
        names = [description[0] for description in cursor.description]
        text = "; ".join(names) + "\n"
        for row in rows:
            row = [str(i) for i in row]
            text += "; ".join(row) + "\n"
    finally:
        conn.close()

    return text

Let's first check that functions work on its own, without agent

In [3]:
tables = db_query("SELECT name FROM sqlite_master WHERE type='table';")
print("Tables:", tables)


Tables: name
lifespan_change
gene_criteria
gene_hallmarks
longevity_associations



## Making a naive sql agent

Now we will create an agent, It will handle simple tasks.

First, let's install the just-agent library with pip and set up API key for Groq

In [4]:
!pip install just-agents-core==0.7.3
!pip install just-agents-tools==0.7.3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.1/80.1 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 64.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.2/243.2 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 647.0/647.0 kB 36.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 5.0 MB/s eta 0:00:00
  Attempting uninstall: rich
    Found existing installation: rich 13.9.4
    Uninstalling rich-13.9.4:
      Successfully uninstalled rich-13.9.4
  Attempting uninstall: openai
    Found existing installation: openai 1.78.1
    Uninstalling openai-1.78.1:
      Successfully uninstalled openai-1.78.1
ERROR: pip's dependency resolver does not currently take into accou

In [12]:
import os
import getpass
from dotenv import load_dotenv

def get_api_keys():
  """Gets API keys from the user."""
  load_dotenv(override=True)

  # Check if GROQ_API_KEY is already set in environment
  groq_api_key = os.environ.get("GROQ_API_KEY")
  gemini_api_key = os.environ.get("GEMINI_API_KEY")

  # Only prompt for key if not already set
  if not groq_api_key:
    print("Please enter your API keys:")
    groq_api_key = getpass.getpass("GROQ_API_KEY: ")
    # Set environment variable
    os.environ["GROQ_API_KEY"] = groq_api_key
    print("GROQ_API key set successfully.")
  else:
    print("GROQ API key already set in environment.")


  # Only prompt for key if not already set
  if not gemini_api_key:
    print("Please enter your API keys:")
    gemini_api_key = getpass.getpass("GEMINI_API_KEY: ")
    # Set environment variable
    os.environ["GEMINI_API_KEY"] = gemini_api_key
    print("GEMINI_API key set successfully.")
  else:
    print("GEMINI API key already set in environment.")


# Call the function to get API keys from the user
get_api_keys()

GROQ API key already set in environment.
Please enter your API keys:
GEMINI_API_KEY: ··········
GEMINI_API key set successfully.


The following code just allows for user to see all comands and text outputed in the interface- it is needed to see all that comes out if the output is large.

In [6]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


Then we will create a simple agent that will use the db_query function as a tool and will answer questions which involve the database

Let's ask the agent some simple questions

In [20]:
from just_agents import llm_options
from just_agents.base_agent import BaseAgent




prompt = "What processes are improved in GHR knockout mice?"

session: BaseAgent = BaseAgent(
    llm_options=llm_options.GEMINI_2_FLASH_THINKING_EXP,
    tools=[db_query]
)


result = session.query(prompt)

session.memory.pretty_print_all_messages()

╭─────────────────────────────────────────────────── [System] ────────────────────────────────────────────────────╮
│ You are a helpful AI assistant                                                                                  │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────────── [User] ─────────────────────────────────────────────────────╮
│ What processes are improved in GHR knockout mice?                                                               │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── [Assistant] ──────────────────────────────────────────────────╮
│ Tool Call to db_query:                                                                                          │
│ {"sql": "SELECT phenotype FROM open_genes WHERE gene = 'GHR' AND mutation_type = 'knockout' AND effect =        │
│ 'improved'"}                                                                                                    │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────────── [Tool] ─────────────────────────────────────────────────────╮
│ Response from db_query:                                                                                         │
│ Error occurred during call: 'no such table: open_genes'                                                         │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── [Assistant] ──────────────────────────────────────────────────╮
│ I am sorry, but I cannot answer this question as the necessary table is not available.                          │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

It DIDN'T get this question right, but what if we want to know "Interventions on which genes extended mice lifespan most of all?"

In [21]:
from just_agents.base_agent import BaseAgent
from just_agents import llm_options

prompt = "Interventions on which genes extended mice lifespan most of all?"

agent: BaseAgent = BaseAgent(
    llm_options=llm_options.GEMINI_2_FLASH_THINKING_EXP,
    tools=[db_query] )

result = agent.query(prompt)
agent.memory.pretty_print_all_messages()

╭─────────────────────────────────────────────────── [System] ────────────────────────────────────────────────────╮
│ You are a helpful AI assistant                                                                                  │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────────── [User] ─────────────────────────────────────────────────────╮
│ Interventions on which genes extended mice lifespan most of all?                                                │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── [Assistant] ──────────────────────────────────────────────────╮
│ Tool Call to db_query:                                                                                          │
│ {"sql": "SELECT gene, lifespan_extension FROM open_genes WHERE species = 'mouse' AND intervention = 'genetic'   │
│ ORDER BY lifespan_extension DESC LIMIT 10;"}                                                                    │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────────── [Tool] ─────────────────────────────────────────────────────╮
│ Response from db_query:                                                                                         │
│ Error occurred during call: 'no such table: open_genes'                                                         │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── [Assistant] ──────────────────────────────────────────────────╮
│ I'm sorry, but I encountered an error when trying to query the database. It seems the table `open_genes` does   │
│ not exist. Therefore, I cannot retrieve the information about which genetic interventions extended mice         │
│ lifespan the most at this time.                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

As you see the SQL returned zero results although we know that the data exist by browsing https://open-genes.com/ and/or sqlite file. Let's modify the system prompt to get it right

But how to make it better? simply call the LLMSession with a prompt.

In [19]:
from just_agents.base_agent import BaseAgent
from just_agents import llm_options

prompt = "Interventions on which genes extended mice lifespan most of all? Search all the relevant tables in the open-genes sqlite and only for mouse"

agent: BaseAgent = BaseAgent(
    llm_options=llm_options.GEMINI_2_5_FLASH,
    system_prompt='open_genes.txt',
    tools=[db_query]
)


result = agent.query(prompt)
agent.memory.pretty_print_all_messages()

╭─────────────────────────────────────────────────── [System] ────────────────────────────────────────────────────╮
│ open_genes.txt                                                                                                  │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────────── [User] ─────────────────────────────────────────────────────╮
│ Interventions on which genes extended mice lifespan most of all? Search all the relevant tables in the          │
│ open-genes sqlite and only for mouse                                                                            │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── [Assistant] ──────────────────────────────────────────────────╮
│ Tool Call to db_query:                                                                                          │
│ {"sql": "SELECT name FROM sqlite_master WHERE type='table';"}                                                   │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────────── [Tool] ─────────────────────────────────────────────────────╮
│ Response from db_query:                                                                                         │
│ name                                                                                                            │
│ lifespan_change                                                                                                 │
│ gene_criteria                                                                                                   │
│ gene_hallmarks                                                                                                  │
│ longevity_associations                                                                                          │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── [Assistant] ──────────────────────────────────────────────────╮
│ Tool Call to db_query:                                                                                          │
│ {"sql": "PRAGMA table_info(lifespan_change);"}                                                                  │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭──────────────────────────────────────────────────── [Tool] ─────────────────────────────────────────────────────╮
│ Response from db_query:                                                                                         │
│ model_organism                                                                                                  │
│ mouse                                                                                                           │
│ roundworm Caenorhabditis elegans                                                                                │
│ fly Drosophila melanogaster                                                                                     │
│ rabbit                                                                                                          │
│ rat                                                                                                             │
│ acyrthosiphon pisum                                                                                             │
│ yeasts                                                                                                          │
│ fish Nothobranchius furzeri                                                                                     │
│ fungus Podospora anserina                                                                                       │
│ hamster                                                                                                         │
│ zebrafish                                                                                                       │
│ fish Nothobranchius guentheri                                                                                   │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────────── [Tool] ─────────────────────────────────────────────────────╮
│ Response from db_query:                                                                                         │
│ effect_on_lifespan                                                                                              │
│ increases lifespan                                                                                              │
│ no change                                                                                                       │
│ decreases lifespan                                                                                              │
│ increases lifespan in animals with decreased lifespans                                                          │
│ decreases survival under stress conditions                                                                      │
│ improves survival under stress conditions                                                                       │
│ decreases life span in animals with increased lifespans                                                         │
│ no change under stress conditions                                                                               │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── [Assistant] ──────────────────────────────────────────────────╮
│ Tool Call to db_query:                                                                                          │
│ {"sql": "SELECT HGNC, lifespan_percent_change_mean FROM lifespan_change WHERE model_organism = 'mouse' AND      │
│ effect_on_lifespan = 'increases lifespan' ORDER BY lifespan_percent_change_mean DESC LIMIT 10;"}                │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────────── [Tool] ─────────────────────────────────────────────────────╮
│ Response from db_query:                                                                                         │
│ HGNC; lifespan_percent_change_mean                                                                              │
│ GHR; 68.2                                                                                                       │
│ PROP1; 68.0                                                                                                     │
│ GHR; 55.0                                                                                                       │
│ VEGFA; 51.7                                                                                                     │
│ PROP1; 49.0                                                                                                     │
│ POU1F1; 48.0                                                                                                    │
│ POU1F1; 42.0                                                                                                    │
│ POU1F1; 42.0                                                                                                    │
│ VEGFA; 41.9                                                                                                     │
│ UCP1; 40.7                                                                                                      │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── [Assistant] ──────────────────────────────────────────────────╮
│ Based on the data in the `lifespan_change` table for mice where the intervention increased lifespan, the genes  │
│ associated with the largest mean percentage lifespan extensions are:                                            │
│                                                                                                                 │
│ 1.  **GHR**: 68.2%                                                                                              │
│ 2.  **PROP1**: 68.0%                                                                                            │
│ 3.  **GHR**: 55.0%                                                                                              │
│ 4.  **VEGFA**: 51.7%                                                                                            │
│ 5.  **PROP1**: 49.0%                                                                                            │
│ 6.  **POU1F1**: 48.0%                                                                                           │
│ 7.  **POU1F1**: 42.0%                                                                                           │
│ 8.  **POU1F1**: 42.0%                                                                                           │
│ 9.  **VEGFA**: 41.9%                                                                                            │
│ 10. **UCP1**: 40.7%                                                                                             │
│                                                                                                                 │
│ These are the top 10 genes based on the mean percentage change in lifespan observed in mouse studies.           │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Now to check it let's call the same code but with model LLAMA4_SCOUT

In [14]:
from just_agents.base_agent import BaseAgent
from just_agents import llm_options

prompt = "Interventions on which genes extended mice lifespan most of all? Search all the relevant tables in the open-genes sqlite and only for mouse"


session: BaseAgent = BaseAgent(
    llm_options=llm_options.LLAMA4_SCOUT,
    system_prompt='open_genes.txt',
    tools=[db_query]
)

result = agent.query(prompt)
agent.memory.pretty_print_all_messages()


╭─────────────────────────────────────────────────── [System] ────────────────────────────────────────────────────╮
│ open_genes.txt                                                                                                  │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────────── [User] ─────────────────────────────────────────────────────╮
│ Interventions on which genes extended mice lifespan most of all? Search all the relevant tables in the          │
│ open-genes sqlite and only for mouse                                                                            │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── [Assistant] ──────────────────────────────────────────────────╮
│ Tool Call to db_query:                                                                                          │
│ {"sql": "SELECT name FROM sqlite_master WHERE type='table';"}                                                   │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────────── [Tool] ─────────────────────────────────────────────────────╮
│ Response from db_query:                                                                                         │
│ name                                                                                                            │
│ lifespan_change                                                                                                 │
│ gene_criteria                                                                                                   │
│ gene_hallmarks                                                                                                  │
│ longevity_associations                                                                                          │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── [Assistant] ──────────────────────────────────────────────────╮
│ Tool Call to db_query:                                                                                          │
│ {"sql": "PRAGMA table_info(lifespan_change);"}                                                                  │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭────────────────────────────────────────────────── [Assistant] ──────────────────────────────────────────────────╮
│ Tool Call to db_query:                                                                                          │
│ {"sql": "SELECT HGNC, lifespan_percent_change_mean FROM lifespan_change WHERE model_organism = 'mouse' AND      │
│ effect_on_lifespan = 'increases lifespan' ORDER BY lifespan_percent_change_mean DESC LIMIT 10;"}                │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────────── [Tool] ─────────────────────────────────────────────────────╮
│ Response from db_query:                                                                                         │
│ HGNC; lifespan_percent_change_mean                                                                              │
│ GHR; 68.2                                                                                                       │
│ PROP1; 68.0                                                                                                     │
│ GHR; 55.0                                                                                                       │
│ VEGFA; 51.7                                                                                                     │
│ PROP1; 49.0                                                                                                     │
│ POU1F1; 48.0                                                                                                    │
│ POU1F1; 42.0                                                                                                    │
│ POU1F1; 42.0                                                                                                    │
│ VEGFA; 41.9                                                                                                     │
│ UCP1; 40.7                                                                                                      │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── [Assistant] ──────────────────────────────────────────────────╮
│ Based on the data in the `lifespan_change` table for mice where the intervention increased lifespan, the genes  │
│ associated with the largest mean percentage lifespan extensions are:                                            │
│                                                                                                                 │
│ 1.  **GHR**: 68.2%                                                                                              │
│ 2.  **PROP1**: 68.0%                                                                                            │
│ 3.  **GHR**: 55.0%                                                                                              │
│ 4.  **VEGFA**: 51.7%                                                                                            │
│ 5.  **PROP1**: 49.0%                                                                                            │
│ 6.  **POU1F1**: 48.0%                                                                                           │
│ 7.  **POU1F1**: 42.0%                                                                                           │
│ 8.  **POU1F1**: 42.0%                                                                                           │
│ 9.  **VEGFA**: 41.9%                                                                                            │
│ 10. **UCP1**: 40.7%                                                                                             │
│                                                                                                                 │
│ These are the top 10 genes based on the mean percentage change in lifespan observed in mouse studies.           │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────────── [User] ─────────────────────────────────────────────────────╮
│ Interventions on which genes extended mice lifespan most of all? Search all the relevant tables in the          │
│ open-genes sqlite and only for mouse                                                                            │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── [Assistant] ──────────────────────────────────────────────────╮
│ Based on my previous search in the `lifespan_change` table for mouse studies where interventions increased      │
│ lifespan, the genes associated with the largest mean percentage lifespan extensions are:                        │
│                                                                                                                 │
│ 1.  **GHR**: 68.2%                                                                                              │
│ 2.  **PROP1**: 68.0%                                                                                            │
│ 3.  **GHR**: 55.0%                                                                                              │
│ 4.  **VEGFA**: 51.7%                                                                                            │
│ 5.  **PROP1**: 49.0%                                                                                            │
│ 6.  **POU1F1**: 48.0%                                                                                           │
│ 7.  **POU1F1**: 42.0%                                                                                           │
│ 8.  **POU1F1**: 42.0%                                                                                           │
│ 9.  **VEGFA**: 41.9%                                                                                            │
│ 10. **UCP1**: 40.7%                                                                                             │
│                                                                                                                 │
│ These are the top genes based on the mean percentage change in lifespan observed in the database for mouse      │
│ interventions.                                                                                                  │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

This shows that even with careful arragement prompts can still fail. It is important to check the results and stability of results before anything else.

Here are some more questions to be tested

1. What genes need to be downregulated in worms to extend their lifespan?
2. What processes are improved in GHR knockout mice?
3. Which genetic intervention led to the greatest increase in lifespan in flies?
4. To what extent did the lifespan increase in mice overexpressing VEGFA?
5. Are there any liver-specific interventions that increase lifespan in mice?
6. Which gene-longevity association is confirmed by the greatest number of studies?
7. What polymorphisms in FOXO3 are associated with human longevity?
8. In which ethnic groups was the association of the APOE gene with longevity shown?
9. Is the INS gene polymorphism associated with longevity?
10. What genes are associated with transcriptional alterations?
11. Which hallmarks are associated with the KL gene?
12. What genes change their expression with aging in humans?
13. How many genes are associated with longevity in humans?
14. What types of studies have been conducted on the IGF1R gene?
15. What evidence of the link between PTEN and aging do you know?
16. What genes are associated with both longevity and altered expression in aged humans?
17. Is the expression of the ACE2 gene altered with aging in humans?
18. Interventions on which genes extended mice lifespan most of all?
19. Which knockdowns were most lifespan extending on model animals?